# BERT Word Embedding

In [25]:
from transformers import BertModel, BertTokenizer
import numpy as np
import streamlit as st
import re
import pandas as pd
from datetime import datetime
import nltk
import torch

In [63]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [139]:




df = pd.read_csv('../../data/processed/light.csv')
# Filter
timestamps = df.year.to_list()
texts = df.text.to_list()

text = texts[1]


It privilege express , Mr. President, congratulations Afghanistan delegation election, justly unanimously voted Assembly. It also privilege extend fellow representatives greetings Royal Afghan Government, well sincerest wishes success current session General Assembly.Our attachment United Nations Charter principles complete adherence principles human rights self-determination peoples based ideological grounds also result long experience free small country controversial events modern history. We believe peace world can secured bases, certain future prosperity depends peace. In saying , posing moralists , contrary, humility expressing conviction fellow Member States attached principles .The attitude Afghanistan delegation previous sessions Assembly inspired principles, will continue case current session. This attitude characterized kind opposition towards country group countries, idea different basic ideal United Nations , distinction among Member States based geographical considerations

In [91]:
marked_text = "[CLS] " + text + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'. format(tup[0], tup[1]))

['[CLS]', 'it', 'privilege', 'express', ',', 'mr', '.', 'president', ',', 'congratulations', 'afghanistan', 'delegation', 'election', ',', 'just', '##ly', 'unanimously', 'voted', 'assembly', '.', 'it', 'also', 'privilege', 'extend', 'fellow', 'representatives', 'greeting', '##s', 'royal', 'afghan', 'government', ',', 'well', 'sincere', '##st', 'wishes', 'success', 'current', 'session', 'general', 'assembly', '.', 'our', 'attachment', 'united', 'nations', 'charter', 'principles', 'complete', 'adherence', 'principles', 'human', 'rights', 'self', '-', 'determination', 'peoples', 'based', 'ideological', 'grounds', 'also', 'result', 'long', 'experience', 'free', 'small', 'country', 'controversial', 'events', 'modern', 'history', '.', 'we', 'believe', 'peace', 'world', 'can', 'secured', 'bases', ',', 'certain', 'future', 'prosperity', 'depends', 'peace', '.', 'in', 'saying', ',', 'posing', 'moral', '##ists', ',', 'contrary', ',', 'hum', '##ility', 'expressing', 'conviction', 'fellow', 'membe

In [201]:
pd_words=pd.Series(tokenized_text)
print(pd_words=="the")
print(pd_words.where(pd_words=="the"))

list_index = list(pd_words.index[pd_words=="the"])


len(list_words)

0      False
1      False
2      False
3      False
4      False
       ...  
609    False
610    False
611    False
612    False
613    False
Length: 614, dtype: bool
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
609    NaN
610    NaN
611    NaN
612    NaN
613    NaN
Length: 614, dtype: object


614

In [92]:
segments_ids = [1] * len(tokenized_text)


In [141]:
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor = tokens_tensor[:,0:512]
segments_tensors = torch.tensor([segments_ids])
segments_tensors = segments_tensors[:,0:512]

model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True)
model.eval()

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    
    

In [142]:
tokens_tensor.shape

torch.Size([1, 512])

In [144]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the 
t is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 512
      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 512, 768])


In [158]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings.size()

token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()



torch.Size([13, 512, 768])

In [162]:
token_embeddings = token_embeddings.permute(1, 0, 2)
print(token_embeddings.size())

torch.Size([512, 13, 768])


In [183]:
token_embeddings_subset = token_embeddings[:,-1, :].squeeze()

In [188]:
token_embeddings_subset[list_words.index("the"), :].shape

torch.Size([768])

In [208]:
list_index.pop()

token_embeddings_subset[list_index]

SyntaxError: unmatched ']' (239714737.py, line 3)

In [175]:
print(token_embeddings)

tensor([[[ 1.6855e-01, -2.8577e-01, -3.2613e-01,  ..., -2.7571e-02,
           3.8253e-02,  1.6400e-01],
         [ 3.5559e-02, -4.6038e-03, -1.6089e-01,  ...,  2.1338e-01,
          -7.7714e-02,  7.0369e-02],
         [ 5.8702e-02, -1.8409e-01, -2.6115e-01,  ...,  4.8097e-01,
          -1.8088e-02, -3.7339e-02],
         ...,
         [ 1.5614e-02, -8.0550e-01,  1.2749e+00,  ..., -1.4153e+00,
           3.9181e-01, -8.6010e-01],
         [-1.3222e-01, -8.0118e-01,  7.2541e-01,  ..., -1.2620e+00,
           1.2361e-01, -8.2128e-01],
         [ 1.5054e-01, -2.5000e-01,  3.7395e-01,  ..., -5.4205e-01,
           8.6377e-02, -5.3121e-01]],

        [[-4.2272e-01, -2.8894e-02, -1.4556e-01,  ...,  7.1141e-01,
           9.9297e-01,  1.9780e-01],
         [-4.3760e-01, -5.4480e-01, -9.2192e-04,  ...,  5.0488e-01,
           7.6371e-01,  6.4694e-01],
         [-4.9401e-01, -6.1803e-01, -1.4912e-01,  ...,  7.5252e-01,
           5.2793e-01,  6.6038e-01],
         ...,
         [-9.2898e-01,  7

In [163]:
token_vecs_cat = []
for token in token_embeddings:
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vecs_cat.append(cat_vec)
    
print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 512 x 3072


In [164]:

token_vecs_sum = []


# For each token in the sentence...
for token in token_embeddings:

    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
token_vecs = hidden_states[-2][0]
sentence_embedding = torch.mean(token_vecs, dim=0)

for i, token_str in enumerate(tokenized_text):
    print(i, token_str)

Shape is: 512 x 768
0 [CLS]
1 it
2 privilege
3 express
4 ,
5 mr
6 .
7 president
8 ,
9 congratulations
10 afghanistan
11 delegation
12 election
13 ,
14 just
15 ##ly
16 unanimously
17 voted
18 assembly
19 .
20 it
21 also
22 privilege
23 extend
24 fellow
25 representatives
26 greeting
27 ##s
28 royal
29 afghan
30 government
31 ,
32 well
33 sincere
34 ##st
35 wishes
36 success
37 current
38 session
39 general
40 assembly
41 .
42 our
43 attachment
44 united
45 nations
46 charter
47 principles
48 complete
49 adherence
50 principles
51 human
52 rights
53 self
54 -
55 determination
56 peoples
57 based
58 ideological
59 grounds
60 also
61 result
62 long
63 experience
64 free
65 small
66 country
67 controversial
68 events
69 modern
70 history
71 .
72 we
73 believe
74 peace
75 world
76 can
77 secured
78 bases
79 ,
80 certain
81 future
82 prosperity
83 depends
84 peace
85 .
86 in
87 saying
88 ,
89 posing
90 moral
91 ##ists
92 ,
93 contrary
94 ,
95 hum
96 ##ility
97 expressing
98 conviction
99 fell

In [202]:
print(len(token_vecs_sum))
list_index

print('First 5 vector values for each instance of "keywords".')
print('')
print("first the   ", str(list_index[1][:5]))
print("second the  ", str(list_index[2][:5]))

512
First 5 vector values for each instance of "keywords".



TypeError: 'int' object is not subscriptable

In [174]:
len(token_vecs_sum)

512

In [30]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [76]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        #print(outputs[1:])
        #original code was 
        hidden_states = outputs[2]
        print(len(outputs))
 
        
    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    token_embeddings.size()
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
    
    
    return list_token_embeddings

In [83]:

for text in texts: 
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)


3


RuntimeError: The expanded size of the tensor (614) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 614].  Tensor sizes: [1, 512]

In [82]:
tokens_vecs_cat = []
for token in list_token_embeddings:
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    tokens_vecs_cat.append(cat_vec)
print( 'Shape is: %d x %d' % (len(tokens_vecs_cat), len(tokens_vecs_cat[0])))

NameError: name 'token_embeddings' is not defined

In [52]:
from scipy.spatial.distance import cosine

# Calculating the distance between the
# embeddings of 'bank' in all the
# given contexts of the word

list_of_distances = []
for text1, embed1 in zip(texts, target_word_embeddings):
    for text2, embed2 in zip(texts, target_word_embeddings):
        cos_dist = 1 - cosine(embed1, embed2)
        list_of_distances.append([text1, text2, cos_dist])

distances_df = pd.DataFrame(list_of_distances, columns=['text1', 'text2', 'distance'])

In [54]:
distances_df[distances_df.text1 == 'world']

,text1,text2,distance


In [ ]:
cos_dist = 1 - cosine(target_word_embeddings[0], np.sum(target_word_embeddings, axis=0))
print(f'Distance between context-free and context-averaged = {cos_dist}')